# First Try with a sequential Neural Network

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor

from functions import *
from data_loading import load_data

Heavily inspired by [this tutorial](https://towardsdatascience.com/regression-based-neural-networks-with-tensorflow-v2-0-predicting-average-daily-rates-e20fffa7ac9a)

In [ ]:
flights_test, flights_train = load_data()

In [ ]:
origin_features = [
            'initial_delay', 
            'initial_delay_mean',
            # 'ORIGIN_AIRPORT',
            # 'DESTINATION_AIRPORT',
            'DISTANCE', 
            'DAY',
            'DAY_OF_WEEK',
            'MONTH',
            'ARRIVAL_DELAY'
            ]

features_test = np.array([a for a in origin_features if a != 'ARRIVAL_DELAY'])
submission_set = test_set[features_test]

# only do this if you got categorical features
# submission_set = pd.get_dummies(submission_set)

arrival_delay = 'ARRIVAL_DELAY'

# if the full set should be used for training
# training_subset = training_set[origin_features]
training_subset = training_set.sample(300000)[origin_features]

# separating the variable that is to be predicted by the model from the rest
X = training_subset.drop(arrival_delay, axis=1)
y = training_subset[arrival_delay]

# simple way of one-hot encoding (if categorical features are in the training set)
# X = pd.get_dummies(X)
# X = X[submission_set.columns]

features = X.columns

# built a validation set on flights_train
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

print(y_train.shape)
y_train = np.reshape([y_train], (-1,1))

print(y_val.shape)
y_val = np.reshape([y_val], (-1,1))

print(scaler_x.fit(X_train))
X_train_scale=scaler_x.transform(X_train)

print(scaler_y.fit(y_train))
y_train_scale=scaler_y.transform(y_train)



print(scaler_x.fit(X_val))
X_val_scale=scaler_x.transform(X_val)

print(scaler_y.fit(y_val))
y_val_scale=scaler_y.transform(y_val)

## Creation of Model

In [ ]:
model = Sequential()

# input_dim should be the same as the amount of features
model.add(Dense(12, input_dim=len(features_test), kernel_initializer='normal', activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='linear'))
model.summary()


In [ ]:

model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

## Training of Model

In [ ]:
history = model.fit(X_train_scale, y_train_scale, epochs=80, batch_size=50,  verbose=1, validation_split=0.2)

## Visualisation of Learning

In [ ]:
print(history.history.keys())
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Create Prediction

In [ ]:
y_pred_scale = model.predict(X_val_scale)
# invert normalize
y_val_inv = scaler_y.inverse_transform(y_val_scale) 
y_pred_inv = scaler_y.inverse_transform(y_pred_scale) 
X_val_inv = scaler_x.inverse_transform(X_val_scale)

print(range(0, len(y_val)))
predictions = pd.DataFrame({"actual": y_val_inv.reshape(1, -1)[0], "predicted": y_pred_inv.reshape(1, -1)[0]}, index=range(0, len(y_val)))

mse(predictions['actual'], predictions['predicted'])